# 멀티 에이전트 협업 네트워크

이 튜토리얼에서는 **멀티 에이전트 네트워크**를 LangGraph를 활용하여 구현하는 방법을 다룹니다. 멀티 에이전트 네트워크는 복잡한 작업을 여러 개의 전문화된 에이전트들로 나누어 처리하는 "분할 정복" 접근 방식을 사용하는 아키텍처입니다. 

이를 통해 단일 에이전트가 많은 도구를 비효율적으로 사용하는 문제를 해결하고, 각 에이전트가 자신의 전문 분야에서 효과적으로 문제를 해결하도록 합니다.

> **참고 문서**: [LangChain Multi-Agent](https://docs.langchain.com/oss/python/langchain/multi-agent/)

본 튜토리얼은 AutoGen 논문에서 영감을 받아, LangGraph를 활용하여 이러한 멀티 에이전트 네트워크를 구축하는 방법을 단계별로 살펴봅니다.

---

## 왜 멀티 에이전트 네트워크인가?

단일 에이전트는 특정 도메인 내에서 일정 수의 도구를 사용할 때 효율적일 수 있습니다. 그러나 한 에이전트가 너무 많은 도구를 다루면 다음과 같은 문제가 발생합니다.

1. 도구 사용 로직이 복잡해집니다.
2. 에이전트가 한 번에 처리해야 할 정보 양이 증가하여 비효율적입니다.

"분할 정복" 접근을 사용하면 각 에이전트는 특정 업무나 전문성 영역에 집중할 수 있고, 전체 작업이 네트워크 형태로 나뉘어 처리됩니다. 각 에이전트는 자신이 잘하는 일을 처리하고, 필요 시 해당 업무를 다른 전문 에이전트에게 위임하거나 도구를 적절히 활용합니다.

---

## 주요 내용

이 튜토리얼에서는 다음 내용을 학습합니다.

- **에이전트 생성**: 에이전트를 정의하고, 이를 LangGraph 그래프의 노드로 설정하는 방법
- **도구 정의**: 에이전트가 사용할 도구를 정의하고 노드로 추가하는 방법
- **그래프 생성**: 에이전트와 도구를 연결하여 멀티 에이전트 네트워크 그래프를 구성하는 방법
- **상태 정의**: 그래프 상태를 정의하고, 각 에이전트의 동작에 필요한 상태 정보를 관리하는 방법
- **엣지 로직 정의**: 에이전트 결과에 따라 다른 에이전트나 도구로 분기하는 로직을 설정하는 방법
- **그래프 실행**: 구성된 그래프를 호출하고 실제 작업을 수행하는 방법

## 환경 설정

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다.
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Tutorial")

## 모델 설정

이번 에이전트에 사용할 모델명을 지정합니다. `langchain_teddynote` 패키지의 `get_model_name` 함수를 사용하여 최신 모델명을 가져옵니다.

아래 코드는 GPT-4o 모델을 사용하도록 설정합니다.

In [ ]:
from langchain_teddynote.models import get_model_name, LLMs

# 최신 모델 이름 가져오기
MODEL_NAME = get_model_name(LLMs.GPT4o)

print(f"사용 모델: {MODEL_NAME}")

## 상태 정의

멀티 에이전트 시스템에서 에이전트들이 정보를 공유하기 위한 상태(State)를 정의합니다. `messages`는 에이전트 간 공유하는 메시지 목록이며, `sender`는 마지막 메시지의 발신자를 나타냅니다.

`operator.add`를 사용하면 새 메시지가 기존 메시지 목록에 추가되는 방식으로 상태가 업데이트됩니다. 아래 코드는 에이전트 상태 스키마를 정의합니다.

In [ ]:
import operator
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage


# 상태 정의
class AgentState(TypedDict):
    # 에이전트 간 공유하는 메시지 목록 (새 메시지가 기존 목록에 추가됨)
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # 마지막 메시지의 발신자
    sender: Annotated[str, "The sender of the last message"]

## 도구 정의

에이전트가 사용할 도구들을 정의합니다. 이 튜토리얼에서는 두 가지 도구를 사용합니다.

- **TavilySearch**: 인터넷에서 정보를 검색하는 도구입니다. Research Agent가 필요한 정보를 검색할 때 사용합니다.
- **PythonREPL**: Python 코드를 실행하는 도구입니다. Chart Generator Agent가 차트를 생성할 때 사용합니다.

아래 코드는 Tavily 검색 도구와 Python REPL 도구를 정의합니다.

In [ ]:
from typing import Annotated

from langchain_teddynote.tools.tavily import TavilySearch
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

# Tavily 검색 도구 정의 (최대 5개 결과 반환)
tavily_tool = TavilySearch(max_results=5)

# Python 코드를 실행하는 도구 정의
python_repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Python 코드를 실행합니다. 값의 출력을 보려면 print(...)를 사용해야 합니다."""
    try:
        # 주어진 코드를 Python REPL에서 실행하고 결과 반환
        result = python_repl.run(code)
    except BaseException as e:
        return f"Failed to execute code. Error: {repr(e)}"
    # 실행 성공 시 결과와 함께 성공 메시지 반환
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

## 에이전트 생성

### Research Agent

Research Agent는 `TavilySearch` 도구를 사용하여 웹에서 정보를 검색하는 에이전트입니다. 이 에이전트는 필요한 정보를 리서치하는 역할을 담당합니다.

`create_react_agent`는 ReAct(Reasoning + Acting) 패턴을 따르는 에이전트를 생성하는 함수입니다. 에이전트는 도구를 사용하여 작업을 수행하고, 다른 에이전트와 협업할 수 있습니다.

먼저 시스템 프롬프트를 생성하는 헬퍼 함수를 정의합니다. 아래 코드는 협업 에이전트를 위한 시스템 프롬프트 생성 함수를 정의합니다.

In [ ]:
def make_system_prompt(suffix: str) -> str:
    """협업 에이전트를 위한 시스템 프롬프트를 생성합니다."""
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState

# LLM 정의
llm = ChatOpenAI(model=MODEL_NAME)

# Research Agent 생성
research_agent = create_react_agent(
    llm,
    tools=[tavily_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)


def research_node(state: MessagesState) -> MessagesState:
    """Research Agent 노드 함수입니다."""
    result = research_agent.invoke(state)

    # 마지막 메시지를 HumanMessage로 변환
    last_message = HumanMessage(
        content=result["messages"][-1].content, name="researcher"
    )
    return {
        # Research Agent의 메시지 목록 반환
        "messages": [last_message],
    }

### Chart Generator Agent

Chart Generator Agent는 `PythonREPL` 도구를 사용하여 차트를 생성하는 에이전트입니다. Research Agent가 수집한 데이터를 기반으로 시각화를 담당합니다.

시스템 프롬프트에 한글 폰트 설정 코드를 포함하여 차트에서 한글이 올바르게 표시되도록 합니다. 아래 코드는 Chart Generator Agent를 생성합니다.

In [ ]:
# 차트 생성 에이전트의 시스템 프롬프트 (한글 폰트 설정 포함)
chart_generator_system_prompt = """
You can only generate charts. You are working with a researcher colleague.
Be sure to use the following font code in your code when generating charts.

##### 폰트 설정 #####
import platform

# OS 판단
current_os = platform.system()

if current_os == "Windows":
    # Windows 환경 폰트 설정
    font_path = "C:/Windows/Fonts/malgun.ttf"  # 맑은 고딕 폰트 경로
    fontprop = fm.FontProperties(fname=font_path, size=12)
    plt.rc("font", family=fontprop.get_name())
elif current_os == "Darwin":  # macOS
    # Mac 환경 폰트 설정
    plt.rcParams["font.family"] = "AppleGothic"
else:  # Linux 등 기타 OS
    # 기본 한글 폰트 설정 시도
    try:
        plt.rcParams["font.family"] = "NanumGothic"
    except:
        print("한글 폰트를 찾을 수 없습니다. 시스템 기본 폰트를 사용합니다.")

##### 마이너스 폰트 깨짐 방지 #####
plt.rcParams["axes.unicode_minus"] = False  # 마이너스 폰트 깨짐 방지
"""

# Chart Generator Agent 생성
chart_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=make_system_prompt(chart_generator_system_prompt),
)

In [ ]:
def chart_node(state: MessagesState) -> MessagesState:
    """Chart Generator Agent 노드 함수입니다."""
    result = chart_agent.invoke(state)

    # 마지막 메시지를 HumanMessage로 변환
    last_message = HumanMessage(
        content=result["messages"][-1].content, name="chart_generator"
    )
    return {
        # Chart Agent의 메시지 목록 반환
        "messages": [last_message],
    }

In [ ]:
from langgraph.graph import END


def router(state: MessagesState):
    """라우터 함수: 에이전트 출력에 따라 다음 노드를 결정합니다."""
    messages = state["messages"]
    last_message = messages[-1]
    if "FINAL ANSWER" in last_message.content:
        # 작업이 완료되면 종료
        return END
    return "continue"

## 그래프 생성

이제 노드를 정의하고 그래프를 구성합니다. Research Agent와 Chart Generator Agent를 노드로 추가하고, 조건부 엣지를 사용하여 에이전트 간의 흐름을 제어합니다.

- **researcher**: Research Agent 노드
- **chart_generator**: Chart Generator Agent 노드
- **조건부 엣지**: 에이전트 출력에 따라 다음 노드를 결정합니다. "FINAL ANSWER"가 포함되면 종료하고, 그렇지 않으면 다른 에이전트로 전달합니다.

아래 코드는 그래프를 구성하고 컴파일합니다.

### 에이전트 노드 및 엣지 정의

그래프에 노드를 추가하고 조건부 엣지를 정의합니다. 아래 코드는 완전한 멀티 에이전트 네트워크 그래프를 구성합니다.

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# 그래프 생성
workflow = StateGraph(MessagesState)

# 노드 추가
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

# 조건부 엣지 추가 (researcher 노드에서)
workflow.add_conditional_edges(
    "researcher",
    router,
    {"continue": "chart_generator", END: END},
)

# 조건부 엣지 추가 (chart_generator 노드에서)
workflow.add_conditional_edges(
    "chart_generator",
    router,
    {"continue": "researcher", END: END},
)

# 시작점 설정
workflow.add_edge(START, "researcher")

# 그래프 컴파일
app = workflow.compile(checkpointer=MemorySaver())

### 그래프 시각화

생성한 그래프의 구조를 시각화하여 확인합니다. `xray=True` 옵션을 사용하면 내부 서브그래프 구조도 함께 표시됩니다. 아래 코드는 그래프를 시각화합니다.

In [ ]:
from langchain_teddynote.graphs import visualize_graph

# 그래프 시각화
visualize_graph(app, xray=True)

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph

# config 설정 (재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = {
    "messages": [
        HumanMessage(
            content="2010년 ~ 2024년까지의 대한민국의 1인당 GDP 추이를 그래프로 시각화 해주세요."
        )
    ],
}

# 그래프 실행
invoke_graph(app, inputs, config, node_names=["researcher", "chart_generator", "agent"])